## Instructions

We first downloaded the files from this link <http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip> 

Then we dragged the pubmed folder into that same folder, with the copied abstracts (if you don't want to re-download them all again).

There are a few files you will need to make sure are present:

`lexparser-gui.bat                  
lexparser-gui.command              
lexparser-gui.sh                   
lexparser-lang-train-test.sh       
lexparser-lang.sh                  
lexparser.bat                      
lexparser.sh                       `

You will also need to add the `edu` folder that can be found here:
<https://www.dropbox.com/s/t9uk4z1xznpo0jz/jars.zip?dl=0>

Add the .zip extension to the `stanford-corenlp-3.8.0-models.jar` file, and unzip it. Copy that `edu` folder and paste it in to your home directory.

## Here is a list of some of the files you should see in your home folder...

In [18]:
!ls

build.xml			lexparser.sh
corenlp.sh			LIBRARY-LICENSES
CoreNLP-to-HTML.xsl		LICENSE.txt
edu				Makefile
ejml-0.23.jar			more_abstracts.json
ejml-0.23-src.zip		parsing_notebook.ipynb
input.txt			patterns
input.txt.xml			pom.xml
javax.json-api-1.0-sources.jar	protobuf.jar
javax.json.jar			pubmed
joda-time-2.9-sources.jar	README.txt
joda-time.jar			SemgrexDemo.java
jollyday-0.4.9-sources.jar	ShiftReduceDemo.java
jollyday.jar			slf4j-api.jar
KeywordSentences.py		slf4j-simple.jar
KeywordSentences.txt		stanford-corenlp-3.8.0.jar
lexparser2.sh			stanford-corenlp-3.8.0-javadoc.jar
lexparser.bat			stanford-corenlp-3.8.0-models.jar
lexparser_caseless.sh		stanford-corenlp-3.8.0-sources.jar
lexparser-gui.bat		StanfordCoreNlpDemo.java
lexparser-gui.command		StanfordDependenciesManual.pdf
lexparser-gui.sh		sutime
lexparser-lang.sh		tokensregex
lexparser-lang-train-test.sh	xom-1.2.10-src.jar
lexparser_rnn.sh		xom.jar


# Parsing the Pubmed Abstracts

In [65]:
import pubmed.utils as pb
import json
from pprint import pprint
# utf-8 support
import codecs
# spit abstracts to sentences
from nltk.tokenize import sent_tokenize

In [22]:
search_term = 'ACE inhibitor'
max_results = 100
query = pb.PubMedQuery(search_term, max_results)

In [23]:
ids = query.id_getter()
print "Num abstracts: ", len(ids)

Num abstracts:  899


## This is to just download a few of the abstracts

In [26]:
%%time
#I just tried to download a few 
# if you don't want to download all abstracts, please do not use the download_all_abstracts() method, but rather the 
# abstract_getter() as below
pb.PubMedQuery.COUNT = 0
max_results = 25
query = pb.PubMedQuery(search_term, max_results)
ids = query.id_getter()
abstracts = query.abstract_getter(ids)

CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 2.12 s


## This is to download a TON of the abstracts

In [70]:
# %%time
# pb.PubMedQuery.COUNT = 0
# max_results = 100
#full_query = pb.download_all_abstracts(search_term, max_results)
#ids = full_query.id_getter()
#abstracts = full_query.id_getter(ids)

## Save abstracts to JSON file

In [30]:
json_file = 'my_ten_abstracts.json'
#json_file = 'more_abstracts.json'
print 'Saving to ' + json_file
with codecs.open(json_file, 'w','utf-8') as outfile:
    json.dump(abstracts, outfile, indent=4)

Saving to my_ten_abstracts.json


## We might not need all the sentences in the abstract, so I am seeing what would happen if we just parse out the ones with key words, and write them to a textfile, and then parse the resulting file.

Keyword selection could happen here in the sentence tokenizer.

In [32]:
sentences = []
with codecs.open('my_ten_abstracts.json','r','utf-8') as data_file:    
    data = json.load(data_file)
    for abstract in data.itervalues():
        sentences.append(sent_tokenize(abstract))
    # flatten the list of abstracts into one long list of sentences
    sentences = [sent for s in sentences for sent in s]
    print "Sentences: ",len(sentences)

Sentences:  234


In [33]:
keyword = 'ACEI'
key_sentences = []
transformed_sentences = pb.ace_substitutor(sentences, 'ACEI')
for sent in transformed_sentences:
    if keyword in sent:
        key_sentences.append(sent)

In [69]:
%%writefile KeywordSentences.py
import json
import codecs
import pubmed.utils as pb

#change keyword to ACE
keyword = 'ACEI'
with codecs.open('my_ten_abstracts.json','r','utf-8') as data_file:    
    data = json.load(data_file)
    #pick snippets related to ACE inhibitors
    for abstract in data.itervalues():
        transformed_abstract = pb.ace_substitutor(abstract, 'ACEI') 
        if keyword in transformed_abstract:
            print transformed_abstract.encode('utf-8') + '\n'



Overwriting KeywordSentences.py


In [70]:
!python2 KeywordSentences.py > KeywordSentences.txt

In [71]:
#This mimics the format that the example parser file has
!head KeywordSentences.txt

renin-angiotensin-aldosterone system (raas) antagonists, including ACEI, angiotensin receptor blockers (arb), and mineralocorticoid receptor antagonists (mra) decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia, especially when used in combination. prevention of hyperkalemia and its associated complications requires careful patient selection, counseling regarding dietary potassium intake, awareness of drug interactions, and regular laboratory surveillance. recent data suggests that the risk of hyperkalemia may be further moderated through the use of combined angiotensin-neprilysin inhibitors, novel mras, and novel potassium binding agents. clinicians should be mindful of the risk of hyperkalemia when prescribing raas inhibitors to patients with heart failure. in patients at highest risk, such as those with diabetes, the elderly, and advanced chronic kidney disease, more intensive laboratory surveillance of potassium and creatinine may be required. no

In [38]:
!chmod a+x lexparser.sh

# Testing with PCFG Model*

*notice I made a change to the lexparser file to allow for more memory

In [39]:
!cat ./lexparser.sh

#!/usr/bin/env bash
#
# Runs the English PCFG parser on one or more files, printing trees only

if [ ! $# -ge 1 ]; then
  echo Usage: `basename $0` 'file(s)'
  echo
  exit
fi

scriptdir=`dirname $0`

java -mx500m -cp "$scriptdir/*:" edu.stanford.nlp.parser.lexparser.LexicalizedParser \
 -outputFormat "penn,typedDependencies" edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz $*


In [72]:
%%timeit
! ./lexparser.sh  KeywordSentences.txt

[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsing [sent. 1 len. 44]: renin-angiotensin-aldosterone system -LRB- raas -RRB- antagonists , including ACEI , angiotensin receptor blockers -LRB- arb -RRB- , and mineralocorticoid receptor antagonists -LRB- mra -RRB- decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia , especially when used in combination .
(ROOT
  (S
    (NP
      (NP (JJ renin-angiotensin-aldosterone) (NN system)
        (PRN (-LRB- -LRB-)
          (NP (NNS raas))
          (-RRB- -RRB-))
        (NNS antagonists))
      (, ,)
      (PP (VBG including)
        (NP (NNP ACEI)))
      (, ,))
    (VP (VBD angiotensin)
      (NP
        (NP
          (NP (NN receptor) (NNS blockers))
          (PRN (-LRB- -LRB-)
            (NP (NN arb))
            (-RRB- -RRB-)))
        (, 

## Testing with RNN Model

In [14]:
!cat ./lexparser_rnn.sh

#!/usr/bin/env bash
#
# Runs the English PCFG parser on one or more files, printing trees only

if [ ! $# -ge 1 ]; then
  echo Usage: `basename $0` 'file(s)'
  echo
  exit
fi

scriptdir=`dirname $0`

java -mx500m -cp "$scriptdir/*:" edu.stanford.nlp.parser.lexparser.LexicalizedParser \
 -outputFormat "penn,typedDependencies" edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz $*


In [15]:
%%timeit
! ./lexparser_rnn.sh  KeywordSentences.txt

[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INFO edu.stanford.nlp.parser.l

## Test with Caseless PCFG  Model

In [30]:
!cat ./lexparser_caseless.sh

#!/usr/bin/env bash
#
# Runs the English PCFG parser on one or more files, printing trees only

if [ ! $# -ge 1 ]; then
  echo Usage: `basename $0` 'file(s)'
  echo
  exit
fi

scriptdir=`dirname $0`

java -mx500m -cp "$scriptdir/*:" edu.stanford.nlp.parser.lexparser.LexicalizedParser \
 -outputFormat "penn,typedDependencies" edu/stanford/nlp/models/lexparser/englishPCFG.caseless.ser.gz $*


In [19]:
%%timeit
! ./lexparser_caseless.sh  KeywordSentences.txt

[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.caseless.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.caseless.ser.gz ... done [0.5 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INFO edu.stanford.nlp.parser.lexparser.LexicalizedParser - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.caseless.ser.gz ... done [0.6 sec].
Parsing file: KeywordSentences.txt
Parsed file: KeywordSentences.txt [0 sentences].
Parsed 0 words in 0 sentences (0.00 wds/sec; 0.00 sents/sec).
[main] INF

## Command Line Sentiment Analysis

This creates an output file with tuples and sentiments! We can modify which annotators we use. I think maybe just the sentiment and the ner is all we need.

In [20]:
!java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref,sentiment -file KeywordSentences.txt -outputFormat text

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [0.8 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.2 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [24]:
!cat *.out >> output_file.json

In [23]:
!cat output_file.json

## Sentences of note:

`Plasma concentrations of AII were significantly decreased by captopril and increased by losartan`.
`nmod:of(concentrations-2, AII-4)`
`nmod:agent(decreased-7, captopril-9)`
`nmod:by(increased-11, losartan-13)`


## Experimenting with Selected Abstracts

In [161]:
#From <https://www.ncbi.nlm.nih.gov/pubmed/28656517>

!echo "Renin-angiotensin-aldosterone system (RAAS) antagonists, including angiotensin-converting enzyme inhibitors (ACEI), angiotensin receptor blockers (ARB), and mineralocorticoid receptor antagonists (MRA) decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia, especially when used in combination. Prevention of hyperkalemia and its associated complications requires careful patient selection, counseling regarding dietary potassium intake, awareness of drug interactions, and regular laboratory surveillance. Recent data suggests that the risk of hyperkalemia may be further moderated through the use of combined angiotensin-neprilysin inhibitors, novel MRAs, and novel potassium binding agents. Clinicians should be mindful of the risk of hyperkalemia when prescribing RAAS inhibitors to patients with heart failure. In patients at highest risk, such as those with diabetes, the elderly, and advanced chronic kidney disease, more intensive laboratory surveillance of potassium and creatinine may be required. Novel therapies hold promise for reducing the risk of hyperkalemia and enhancing the tolerability of RAAS antagonists." > sample_abstract.txt

In [162]:
!head sample_abstract.txt

Renin-angiotensin-aldosterone system (RAAS) antagonists, including angiotensin-converting enzyme inhibitors (ACEI), angiotensin receptor blockers (ARB), and mineralocorticoid receptor antagonists (MRA) decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia, especially when used in combination. Prevention of hyperkalemia and its associated complications requires careful patient selection, counseling regarding dietary potassium intake, awareness of drug interactions, and regular laboratory surveillance. Recent data suggests that the risk of hyperkalemia may be further moderated through the use of combined angiotensin-neprilysin inhibitors, novel MRAs, and novel potassium binding agents. Clinicians should be mindful of the risk of hyperkalemia when prescribing RAAS inhibitors to patients with heart failure. In patients at highest risk, such as those with diabetes, the elderly, and advanced chronic kidney disease, more intensive laboratory surveillance of p

In [163]:
!java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref,sentiment -file sample_abstract.txt -outputFormat text

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.2 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [164]:
!cat sample_abstract.txt.out

Sentence #1 (49 tokens, sentiment: Very negative):
Renin-angiotensin-aldosterone system (RAAS) antagonists, including angiotensin-converting enzyme inhibitors (ACEI), angiotensin receptor blockers (ARB), and mineralocorticoid receptor antagonists (MRA) decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia, especially when used in combination.
[Text=Renin-angiotensin-aldosterone CharacterOffsetBegin=0 CharacterOffsetEnd=29 PartOfSpeech=NN Lemma=renin-angiotensin-aldosterone NamedEntityTag=O SentimentClass=Neutral]
[Text=system CharacterOffsetBegin=30 CharacterOffsetEnd=36 PartOfSpeech=NN Lemma=system NamedEntityTag=O SentimentClass=Neutral]
[Text=-LRB- CharacterOffsetBegin=37 CharacterOffsetEnd=38 PartOfSpeech=-LRB- Lemma=-lrb- NamedEntityTag=O SentimentClass=Neutral]
[Text=RAAS CharacterOffsetBegin=38 CharacterOffsetEnd=42 PartOfSpeech=NN Lemma=raas NamedEntityTag=O SentimentClass=Neutral]
[Text=-RRB- CharacterOffsetBegin=42 CharacterOffsetEnd=43 PartO

Our takeaway sentence is that "ACE Inhibitors (ACEI) [...] decrease mortality and morbidity in heart failure but increase the risk of hyperkalemia...".

`amod(inhibitors-11, angiotensin-converting-9)`

`dobj(increase-38, risk-40)`

`nmod:of(risk-40, hyperkalemia-42)`


## Experimenting with specific drug names instead of a drug class

It didn't look like we had a lot of interactions between drugs and molecules, so I wonder if we change our serach terms a little bit to include the drug names. The search terms are not case sensitive.

`Drugs in class: Captopril, Lisinopril, Ramipril, Benazepril`

In [165]:
search_term = ['Captopril', 'Lisinopril','Ramipril','Benazepril']
max_results = 100
for i in search_term:
    query2 = pb.PubMedQuery(i, max_results)
    
ids2 = query2.id_getter()
print "Num abstracts: ", len(ids2)

abstracts2 = query.abstract_getter(ids2)

json_file2 = 'more_abstracts2.json'
print 'Saving to ' + json_file2
with open(json_file2, 'w') as outfile:
    json.dump(abstracts2, outfile, indent=4)
    
    
sentences2 = []
with open('more_abstracts2.json') as data_file:    
    data = json.load(data_file)
    for abstract in data.itervalues():
        sentences2.append(sent_tokenize(abstract))
    # flatten the list of abstracts into one long list of sentences
    sentences2 = [sent for s in sentences2 for sent in s]
    print "Sentences: ",len(sentences2)

Num abstracts:  899
Saving to more_abstracts2.json
Sentences:  938


In [169]:
#Just looking at some sentences
sentences2[:5]

[u'Benazepril plays an important role in down-regulating the expression of TGFbeta1 and decreasing the accumulation of ECM by blocking intrarenal renin-angiotensin system.',
 u'To study the expression of type I transforming growth factor beta receptor (TGFbetaRI) in renal cortex in streptozotocin-induced diabetic rats and the regulation of benazepril.',
 u'Experimental glomerulosclerosis in rats was induced by adriamycin.',
 u'The treated group was given benazepril (4 mg x kg(-1) x d(-1)).',
 u'Intrarenal angiotensin converting enzyme (ACE) activity and angiotensin II (AngII) concentration were measured with colorimetry and radioimmunoassay respectively.']

In [218]:
%%writefile KeywordSentences2.py
import json

#change keyword to ACE
keywords = ['Captopril', 'Lisinopril','Ramipril','Benazepril']
with open('more_abstracts2.json') as data_file:    
    data = json.load(data_file)
    #pick snippets related to ACE inhibitors
    for i in range(len(data.keys())):
        try:
            if any(k in data[str(i)] for k in keywords):
                print data[str(i)] + '\n'
            else:
                next
        except:
            next
            

Overwriting KeywordSentences2.py


In [219]:
!python KeywordSentences2.py > KeywordSentences2.txt

In [220]:
!cat KeywordSentences2.txt

A number of experimental and clinical investigations support the notion that angiotensin-converting enzyme inhibitor (ACEi) and angiotensin II type 1 receptor blocker (ARB) compounds attenuate renal fibrosis. Fibrosis can be attenuated by either suppressing matrix formation or facilitating matrix degradation. In this study, drugs of ACEi and ARB classes were tested for their ability to facilitate matrix degradation in the kidney. A murine model system in which cyclosporin A (CsA) treatment for a specified period caused interstitial matrix deposition in the kidney was used. CsA was then discontinued, and experimental procedures were initiated to investigate matrix degradation. Benazepril, an ACEi, facilitated matrix degradation via the bradykinin (BK) B2 receptor on tubular epithelial cells in the kidney, whereas CGP-48933, an ARB, did not. In this murine model of CsA nephropathy under ACE blockade, plasminogen activator inhibitor-1 (PAI-1) expression was decreased in tubular epithelial

## Experimenting with Sentiment Analysis

In [222]:
!java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref,sentiment -file KeywordSentences2.txt -outputFormat text

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.0 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [223]:
!cat KeywordSentences2.txt.out

Sentence #1 (32 tokens, sentiment: Negative):
A number of experimental and clinical investigations support the notion that angiotensin-converting enzyme inhibitor (ACEi) and angiotensin II type 1 receptor blocker (ARB) compounds attenuate renal fibrosis.
[Text=A CharacterOffsetBegin=0 CharacterOffsetEnd=1 PartOfSpeech=DT Lemma=a NamedEntityTag=O SentimentClass=Neutral]
[Text=number CharacterOffsetBegin=2 CharacterOffsetEnd=8 PartOfSpeech=NN Lemma=number NamedEntityTag=O SentimentClass=Neutral]
[Text=of CharacterOffsetBegin=9 CharacterOffsetEnd=11 PartOfSpeech=IN Lemma=of NamedEntityTag=O SentimentClass=Neutral]
[Text=experimental CharacterOffsetBegin=12 CharacterOffsetEnd=24 PartOfSpeech=JJ Lemma=experimental NamedEntityTag=O SentimentClass=Neutral]
[Text=and CharacterOffsetBegin=25 CharacterOffsetEnd=28 PartOfSpeech=CC Lemma=and NamedEntityTag=O SentimentClass=Neutral]
[Text=clinical CharacterOffsetBegin=29 CharacterOffsetEnd=37 PartOfSpeech=JJ Lemma=clinical NamedEntityTag=O Sentimen

## It seems like this was a little more focused.